# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../WeatherPy/output_Data/cities.csv', encoding="utf-8")
cities_df.head()

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Bluff,100,NZ,1651962575,98,-46.6000,168.3333,57.79,6.02
1,Alofi,96,NU,1651962575,89,-19.0595,-169.9187,78.69,3.44
2,Kirakira,81,SB,1651962575,80,-10.4544,161.9205,82.76,10.51
3,Manjacaze,0,MZ,1651962533,93,-24.7117,33.8828,64.36,6.06
4,Kavieng,59,PG,1651962387,71,-2.5744,150.7967,84.11,17.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Fill Humidity values and convert to float
humidity = cities_df["Humidity"].astype(float)
max_humidity = humidity.max()

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
humidity = cities_df["Humidity"].tolist()
markers = gmaps.marker_layer(locations,info_box_content=[f"Humidity: {humidity}" for humidity in humidity])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
pref_cities = cities_df.loc[
    (cities_df["Max Temp"] > 60) &
    (cities_df["Max Temp"] < 90) &
    (cities_df["Wind Speed"] < 15) &
    (cities_df["Humidity"] < 75) & 
    (cities_df["Cloudiness"] >= 0) &
    (cities_df["Cloudiness"] < 30)
]
pref_cities = pref_cities.dropna(how='any')
pref_cities.reset_index(inplace=True)
del pref_cities['index']

# Display to output
pref_cities.head()

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,San Rafael,0,AR,1651962285,28,-34.6177,-68.3301,69.19,12.95
1,Mishawaka,0,US,1651962545,30,41.6620,-86.1586,68.47,10.36
2,Kodīnar,0,IN,1651962583,69,20.7903,70.7031,80.69,13.04
3,Jamestown,0,US,1651962585,35,42.0970,-79.2353,62.53,13.80
4,Sakakah,1,SA,1651962586,24,29.9697,40.2064,62.17,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df = pd.DataFrame(pref_cities)
hotel_df

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,San Rafael,0,AR,1651962285,28,-34.6177,-68.3301,69.19,12.95
1,Mishawaka,0,US,1651962545,30,41.6620,-86.1586,68.47,10.36
2,Kodīnar,0,IN,1651962583,69,20.7903,70.7031,80.69,13.04
3,Jamestown,0,US,1651962585,35,42.0970,-79.2353,62.53,13.80
4,Sakakah,1,SA,1651962586,24,29.9697,40.2064,62.17,1.59
5,Mount Isa,0,AU,1651962533,48,-20.7333,139.5000,62.37,8.05
6,Taoudenni,0,ML,1651962599,10,22.6783,-3.9836,87.85,11.65
7,Santa Cruz de la Sierra,20,BO,1651962529,47,-17.8000,-63.1667,78.85,3.44
8,Cidreira,0,BR,1651962608,74,-30.1811,-50.2056,63.01,8.32
9,Mar del Plata,0,AR,1651962487,74,-38.0023,-57.5575,66.94,11.50


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
